In [2]:
# Load data from training data directory, convert from NPZ to tensor, and save to a dict
# containing lift name and tensor data

import os
import torch
import numpy as np

DEADLIFT_TENSORS = {}
BENCH_TENSORS = {}
SQUAT_TENSORS = {}

# Setup paths
script_dir = os.path.dirname(os.path.abspath("model_train.ipynb"))
data_dir = os.path.normpath(os.path.join(script_dir, '..', 'training data'))

# Step 1: Get all subfolder names in data_dir
subfolders = []
for f in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, f)):
        subfolders.append(f)


for subfolder in subfolders:
    subfolder_path = os.path.join(data_dir, subfolder)
    files = []
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)):
            files.append(f)

    #print(f"{subfolder}")
    for file in files:
        data = np.load(os.path.join(subfolder_path, file))

        tensor_dict = {key: torch.tensor(data[key]) for key in data.files}
        if file.split('.')[0].startswith('bench'):
            BENCH_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('deadlift'):
            DEADLIFT_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('squat'):
            SQUAT_TENSORS[file.split('.')[0]] = tensor_dict
        #LIFT_TENSORS[file.split('.')[0]] = tensor_dict
        #print(f" {file.split('.')[0]} loaded")

In [3]:
landmark_indeces_to_labels = {
    0: "nose",
    1: "left eye inner",
    2: "left eye",
    3: "left eye outer",
    4: "right eye inner",
    5: "right eye",
    6: "right eye outer",
    7: "left ear",
    8: "right ear",
    9: "mouth left",
    10: "mouth right",
    11: "left shoulder",
    12: "right shoulder",
    13: "left elbow",
    14: "right elbow",
    15: "left wrist",
    16: "right wrist",
    17: "left pinky",
    18: "right pinky",
    19: "left index",
    20: "right index",
    21: "left thumb",
    22: "right thumb",
    23: "left hip",
    24: "right hip",
    25: "left knee",
    26: "right knee",
    27: "left ankle",
    28: "right ankle",
    29: "left heel",
    30: "right heel",
    31: "left foot index",
    32: "right foot index"
}

In [4]:
# Help understand what is going on in the dicts
for k,v in BENCH_TENSORS.items():
    print(k)
    for key in v.keys():
        print(key)
        try:
            print(f"     {key} contains a {type(v[key])} containing Z coords for {landmark_indeces_to_labels[int(key.split('_')[-1])]} that contains {len(v[key])} points")
        except:
            print(f"     {key} contains a {type(v[key])} containing joint angles for {' '.join(key.split('_')[-2:])} that contains ({len(v[key])} angles)")

bench 2 good lift data
bench 2 good.mp4_landmark_0
     bench 2 good.mp4_landmark_0 contains a <class 'torch.Tensor'> containing Z coords for nose that contains 3 points
bench 2 good.mp4_landmark_1
     bench 2 good.mp4_landmark_1 contains a <class 'torch.Tensor'> containing Z coords for left eye inner that contains 3 points
bench 2 good.mp4_landmark_2
     bench 2 good.mp4_landmark_2 contains a <class 'torch.Tensor'> containing Z coords for left eye that contains 3 points
bench 2 good.mp4_landmark_3
     bench 2 good.mp4_landmark_3 contains a <class 'torch.Tensor'> containing Z coords for left eye outer that contains 3 points
bench 2 good.mp4_landmark_4
     bench 2 good.mp4_landmark_4 contains a <class 'torch.Tensor'> containing Z coords for right eye inner that contains 3 points
bench 2 good.mp4_landmark_5
     bench 2 good.mp4_landmark_5 contains a <class 'torch.Tensor'> containing Z coords for right eye that contains 3 points
bench 2 good.mp4_landmark_6
     bench 2 good.mp4_landm

In [5]:
# for k, v in BENCH_TENSORS.items():
#     for lift, coordinate in v.items():
#         print(lift, "coords",coordinate[2])

for key, value in BENCH_TENSORS.items():
    print(key)
    # for landmark, coordinate in BENCH_TENSORS[key].items():
    #     return (landmark, coordinate[2])

bench 2 good lift data
bench 3 good lift data
bench 4 good lift data
bench 5 good lift data


In [46]:
# Function to Isolate Z Coordinates of a single lift
def getZCoordinates(tensor_dict, key):
    z_coords = {}
    i = 0
    for key, value in tensor_dict[key].items():
        print(key, len(value))
        if i <= 32:
            lift_number_and_landmark = f"{' '.join(key.split(' ')[:2])} {landmark_indeces_to_labels[int(key.split('_')[-1])]}"
            z_coords[lift_number_and_landmark] = value[2]
        i += 1
    return z_coords


In [48]:
# Isolate z coordinates of individual lift for each landmark
# Stores as a dict of dicts

z_coordinates = {}
for k,v in DEADLIFT_TENSORS.items():
    for lift, landmarks in v.items():

        if len(landmarks) == 3:
            lift_z_coords = f"{' '.join(lift.split(' ')[:2])} {landmark_indeces_to_labels[int(' '.join(lift.split('_')[-1:]))]} z coordinates"
            z_coordinates[lift_z_coords] = landmarks[2]

# Example of how to isolate one landmark in a single lift
print(z_coordinates['deadlift 1 nose z coordinates'])

tensor([ 0.3600,  0.3600,  0.3200,  0.3100,  0.2700,  0.3100,  0.2600,  0.2600,
         0.2500,  0.2700,  0.2800,  0.2800,  0.3000,  0.2900,  0.2600,  0.2600,
         0.2700,  0.2400,  0.3100,  0.3200,  0.3400,  0.3000,  0.2800,  0.2500,
         0.2400,  0.1900,  0.2100,  0.2600,  0.2500,  0.2400,  0.2600,  0.3100,
         0.3100,  0.3300,  0.3100,  0.3000, -0.5700, -0.5700, -0.5700, -0.6100,
        -0.6000, -0.5900, -0.5900, -0.5900, -0.5900, -0.5500, -0.5600, -0.5500,
        -0.5600, -0.5600, -0.5700, -0.5800, -0.5800, -0.5800, -0.5700, -0.5600,
        -0.5700, -0.5800, -0.5800, -0.5800, -0.6000, -0.6100, -0.6200, -0.6300,
        -0.6200, -0.6300, -0.6300, -0.6300, -0.6300, -0.6300, -0.6200, -0.6000,
        -0.6100, -0.6200, -0.6300, -0.6300, -0.6300, -0.6200, -0.6300, -0.6400,
        -0.6400, -0.6500, -0.6600, -0.6700, -0.6600, -0.6700, -0.6700, -0.6600,
        -0.6600, -0.6700, -0.6900, -0.6900, -0.6900, -0.6900, -0.6900, -0.6800,
        -0.6600, -0.6500, -0.6500, -0.66